# Medical Data Visualizer

En este proyecto, visualizarás y harás cálculos a partir de datos de exámenes médicos utilizando matplotlib, seaborn y pandas. Los valores del conjunto de datos se recogieron durante los exámenes médicos.

## Descripcion de datos
Las filas del conjunto de datos representan a los pacientes y las columnas representan información como las mediciones corporales, los resultados de varios análisis de sangre y las opciones de estilo de vida. Utilizará el conjunto de datos para explorar la relación entre la enfermedad cardíaca, las medidas corporales, los marcadores sanguíneos y las opciones de estilo de vida.

Nombre del archivo: `medical_examination.csv` el cual tiene la siguiente estructura

| Feature | Variable Type | Variable      | Value Type |
|:-------:|:------------:|:-------------:|:----------:|
| Age | Objective Feature | age | int (days) |
| Height | Objective Feature | height | int (cm) |
| Weight | Objective Feature | weight | float (kg) |
| Gender | Objective Feature | gender | categorical code |
| Systolic blood pressure | Examination Feature | ap_hi | int |
| Diastolic blood pressure | Examination Feature | ap_lo | int |
| Cholesterol | Examination Feature | cholesterol | 1: normal, 2: above normal, 3: well above normal |
| Glucose | Examination Feature | gluc | 1: normal, 2: above normal, 3: well above normal |
| Smoking | Subjective Feature | smoke | binary |
| Alcohol intake | Subjective Feature | alco | binary |
| Physical activity | Subjective Feature | active | binary |
| Presence or absence of cardiovascular disease | Target Variable | cardio | binary |




# Actividades a realizar

Cree un gráfico, en el que se muestren los recuentos de resultados buenos y malos para las variables `cholesterol`, `gluc`, `alco`, `active` y `smoke` para pacientes con cardio=1 y cardio=0 en diferentes paneles.

Usa los datos para completar las siguientes tareas en `medical_data_visualizer.py`:
* Añadir una columna de `overweight` a los datos. Para determinar si una persona tiene sobrepeso, primero calcula su IMC dividiendo su peso en kilogramos por el cuadrado de su altura en metros. Si ese valor es > 25, la persona tiene sobrepeso. Utilice el valor 0 para NO tener sobrepeso y el valor 1 para tenerlo.
* Normalice los datos haciendo que 0 sea siempre bueno y 1 siempre malo. Si el valor de `cholesterol` o `gluc` es 1, haga el valor 0. Si el valor es más de 1, haga el valor 1.
* Convierta los datos en formato "long" y cree un gráfico que muestre los recuentos de valores de las características categóricas utilizando `catplot()` de seaborn. El conjunto de datos debe ser dividido por 'cardio' para que haya un gráfico para cada valor de `cardio`.
* Limpie los datos. Filtre los siguientes segmentos de pacientes que representan datos incorrectos:
  - la presión diastólica es mayor que la sistólica (Mantenga los datos correctos con `(df['ap_lo'] <= df['ap_hi'])`)
  - la altura es inferior al percentil 2,5 (Mantenga los datos correctos con `(df['altura'] >= df['altura'].quantile(0,025))`)
  - la altura es superior al percentil 97,5
  - el peso es inferior al percentil 2,5
  - el peso es superior al percentil 97,5
* Crea una matriz de correlaciones utilizando el conjunto de datos. Grafique la matriz de correlación utilizando `heatmap()` de seaborn. Enmascare el triángulo superior. 

In [1]:

## Librerias necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Importe de datos
df = pd.read_csv("medical_examination.csv")


In [2]:
# Añadiendo columna "overweight" sobrepeso
## Aplicando 1 si es sobrepeso
df["overweight"] = (df["weight"] / (df["height"]/100)**2).apply(
    lambda x:1 if x >25 else 0
    )


In [3]:
# Normalizar los datos haciendo los valores 1 siempre malo y 0 bueno
## Si el valor de cholesterol o gluc es 1, haga el valor 0. 
## Si el valor es más de 1, haga el valor 1.
df["cholesterol"] = df["cholesterol"].apply(lambda x:0 if x==1 else 1)
df["gluc"] = df["gluc"].apply(lambda x:0 if x==1 else 1)

In [4]:
def draw_cat_plot():
    """Crear un dataframe pd.melt solo para los valores de:
         cholesterol, gluc, smoke, alco, active, overweight
    """
    df_cat = pd.melt(df, id_vars=["cardio"], value_vars=[
        "cholesterol", "gluc", "smoke", "alco", "active", "overweight"
        ])
    
    # conjunto de datos divididos por cardio
    df_cat["total"] = 1 # Util para el recuento
    df_cat = df_cat.groupby(
        ["cardio", "variable","value"], as_index=False).count()

    # Grafica de catplot usando sns.catplot()
    fig = sns.catplot(x="variable", y="total", 
                      data=df_cat, hue="value", kind="bar", col="cardio").fig 
    
    fig.savefig("catplot.png")
    return fig

In [5]:

def draw_scatter_plot():
    fig = sns.relplot(x="weight", y="height", 
                      hue="active", style="gender", data=df)
    return fig

In [6]:
def draw_heat_map():
    # tratamiento de datos erroneos o sospechosos
    df_heat = df[
        (df['ap_lo'] <= df['ap_hi']) & 
        (df['height'] >= df['height'].quantile(0.025)) &
        (df['height'] <= df['height'].quantile(0.975)) &
        (df['weight'] >= df['weight'].quantile(0.025)) &
        (df['weight'] <= df['weight'].quantile(0.975)) 
        ] 
    ## Calculo de la matriz de correlaciones
    corr = df_heat.corr(method='pearson')
    
    ## mask para el triangulo superior
    mask = np.triu(corr)
    
    ## Configuracion para la figura
    fig, ax = plt.subplots(figsize=(12, 12))
    
    ## 'dibujar' el heatmap 
    sns.heatmap(
        corr, linewidths=1, annot=True, square=True, 
        mask=mask, fmt='.1f',
        center=0.08, cbar_kws={'shrink':0.5}
        )
    #fig.savefig('heatmap.png')
    
    return fig

# Page View Time Series Visualizer

Para este proyecto visualizarás datos de series temporales utilizando un gráfico de líneas, un gráfico de barras y gráficos de caja. Utilizarás Pandas, Matplotlib y Seaborn para visualizar un conjunto de datos que contiene el número de páginas vistas cada día en el foro de freeCodeCamp.org desde 2016-05-09 hasta 2019-12-03. Las visualizaciones de datos te ayudarán a entender los patrones de las visitas y a identificar el crecimiento anual y mensual




## Actividades a reaizar
* Usa Pandas para importar los datos de `fcc-forum-pageviews.csv`. Establezca el índice en la columna "date".
* Limpiar los datos filtrando los días en los que las páginas vistas estaban en el 2,5% superior del conjunto de datos o en el 2,5% inferior del conjunto de datos.
* Crear una función `draw_line_plot` que utilice Matplotlib para dibujar un gráfico de líneas. El título debería ser "Daily freeCodeCamp Forum Page Views 5/2016-12/2019". La etiqueta en el eje x debe ser "Date" y la etiqueta en el eje y debe ser "Page Views".
* Crear una función `draw_bar_plot` que dibuje un gráfico de barras. Debe mostrar el promedio de páginas vistas diarias para cada mes agrupadas por año. La leyenda debe mostrar las etiquetas de los meses y tener un título de "Months". En el gráfico, la etiqueta del eje x debe ser "Years" y la etiqueta del eje y debe ser "Average Page Views".
* Cree una función `draw_box_plot` que utilice Searborn para dibujar dos gráficos de caja adyacentes. Estos gráficos de caja deben mostrar cómo se distribuyen los valores dentro de un año o mes determinado y cómo se comparan a lo largo del tiempo. El título del primer gráfico debe ser "Year-wise Box Plot (Trend)" y el título del segundo gráfico debe ser "Month-wise Box Plot (Seasonality)", Asegurarse que las etiquetas de los meses en la parte inferior comienzan en "enero" y que los ejes (x,x) están etiquetados correctamente.


In [1]:
# Librerias a usar
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
# Importe de datos, definiendo indice: 'date'
## Confifurando que se pasaran fechas
df = pd.read_csv("fcc-forum-pageviews.csv", 
                 parse_dates=['date'], 
                 index_col='date')

In [ ]:
# Clean data
df = df[
        (df['value'] >= df['value'].quantile(0.025)) &
        (df['value'] <= df['value'].quantile(0.975))
        ]

In [ ]:
def draw_line_plot():
    """Grafico de lineas"""
    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(df.index, df['value'], 'r', linewidth=1)
    ax.set_title('Daily freeCodeCamp Forum Page Views 5/2016-12/2019')
    ax.set_xlabel('Date')
    ax.set_ylabel('Page Views')

    # Save image and return fig (don't change this part)
    #fig.savefig('line_plot.png')
    return fig

In [ ]:
def draw_bar_plot():
   # Draw bar plot
   df['month'] = df.index.month
   df['year'] = df.index.year
   df_bar = df.groupby(['year', 'month'])['value'].mean()
   df_bar = df_bar.unstack()
   
   # Dibuja el grafico de barras
   fig = df_bar.plot.bar(legend=True, figsize=(13,6),
                         ylabel='Average Page Views', xlabel='Years').figure
   
   plt.legend(['January', 'February', 'March', 'April', 'May', 'June', 'July', 
               'August', 'September', 'October', 'November', 'December'])
   
   plt.xticks(fontsize=8)
   plt.yticks(fontsize=8)

   # Save image and return fig (don't change this part)
   fig.savefig('bar_plot.png')
   return fig

In [ ]:
def draw_box_plot():
    # Prepare data for box plots (this part is done!)
    df_box = df.copy()
    df_box.reset_index(inplace=True)
    df_box['year'] = [d.year for d in df_box.date]
    df_box['month'] = [d.strftime('%b') for d in df_box.date]

    # Draw box plots (using Seaborn)
    df_box['month_num'] = df_box['date'].dt.month
    df_box = df_box.sort_values('month_num')
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
    axes[0] = sns.boxplot(x=df_box['year'], y=df_box['value'], ax= axes[0])
    axes[1] = sns.boxplot(x=df_box['month'], y=df_box['value'], ax= axes[1])
    
    axes[0].set_title('Year-wise Box Plot (Trend)')
    axes[0].set_xlabel('Year')
    axes[0].set_ylabel('Page Views')
    
    axes[1].set_title('Month-wise Box Plot (Seasonality)')
    axes[1].set_xlabel('Month')
    axes[1].set_ylabel('Page Views')


    # Save image and return fig (don't change this part)
    fig.savefig('box_plot.png')
    return fig
